# This is the ML Part

So first of all the files need to be loaded from the Data Base

Following is the Calculation of the Influence Score, that needs a swell value. This has to be done by looking up the numbers calculated. 

Third is choosing the number of Clusters

and finally I can Score the Offer types against the Clusters and see how they differ.

In [1]:
import numpy as np
import pandas as pd
import math
import json
from matplotlib import pyplot as plt
import seaborn as sns

from datetime import datetime


import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
from kmodes.kprototypes import KPrototypes



from scipy import interpolate

import sqlite3
from sqlalchemy import create_engine

In [8]:
engine = create_engine('sqlite:///data/data.db')


portfolio = pd.read_sql_table ('portfolio', engine)
transcript = pd.read_sql_table ('transcript', engine)
profile = pd.read_sql_table ('profiles', engine)
received_reaction_matrix = pd.read_sql_table ('received_reaction_matrix', engine)

In [9]:
received_reaction_matrix

,index,person_x,event,time in hours,amount,count,0b1e1539f2cc45b7b9fa7c272da2e1d7,2298d6c36e964ae4a3e7e9706d1fb8c2,2906b810c7d4411798c6938adc9daaa5,3f207df678b143eea3cee63160fa8bed,...,person_y,viewed_yn,viewtoexpire,completed_yn,compacttime,completed_yn_unknown,index_received,index_viewed,index_completed,visits_count
0,0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,0.0,0,0,0,0,0,...,78afa995795e4d85b5d9ceeca43f5fef,1,162,1,126,0,0,15561,47582,0
1,0,e2127556f4f64592b11af22de27a7932,offer received,0,0.0,0,0,0,1,0,...,e2127556f4f64592b11af22de27a7932,1,150,0,0,0,2,20283,-1,0
2,0,389bc3fa690240e798340f5a15918d5c,offer received,0,0.0,0,0,0,0,0,...,389bc3fa690240e798340f5a15918d5c,1,120,1,60,0,5,12650,32625,0
3,0,2eeac8d8feae4a8cad5a6af0499a211d,offer received,0,0.0,0,0,0,0,1,...,2eeac8d8feae4a8cad5a6af0499a211d,0,0,0,0,0,7,-1,-1,0
4,0,aa4862eba776480b8bb9c68455b8c2e1,offer received,0,0.0,0,1,0,0,0,...,aa4862eba776480b8bb9c68455b8c2e1,1,222,0,0,0,8,20284,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56642,0,004c5799adbf42868b9cff0396190900,offer received,576,0.0,0,0,0,0,0,...,004c5799adbf42868b9cff0396190900,1,168,1,-72,0,257855,290020,258606,0
56643,0,e9d127cf3f794ccbaac940f93c15ce8c,offer received,576,0.0,0,0,0,0,1,...,e9d127cf3f794ccbaac940f93c15ce8c,0,0,0,0,0,257858,-1,-1,0
56644,0,670626b55bfb4ba39c85b27cc7cca527,offer received,576,0.0,0,1,0,0,0,...,670626b55bfb4ba39c85b27cc7cca527,1,192,1,42,0,257879,282637,295365,2
56645,0,f924927e7f844ecebf87eef93f4d2688,offer received,576,0.0,0,0,0,0,0,...,f924927e7f844ecebf87eef93f4d2688,1,42,0,0,0,257880,276500,-1,0


In [10]:
def split_on_event (data):

    '''
        Spliting the data set for better performance

    '''

    received = data [data ['event'] == 'offer received']

    viewed = data [data ['event'] == 'offer viewed']

    completed = data [data ['event'] == 'offer completed']

    transactions = data [data ['event'] == 'transaction']

    return received, viewed, completed, transactions


def offer_per_person (offer_type):
    
    reactions = received_reaction_matrix [received_reaction_matrix ['offer_type'] == offer_type]
    offers = received [received ['offer_type'] == offer_type].groupby ('person')['DayOfMonth'].value_counts ().to_frame().unstack().fillna (0)

    auswahl = ['viewed_yn', 'completed_yn',
           'completed_yn_unknown', 'visits_count']



    offers = offers.merge (reactions.groupby ('person_x')[auswahl].sum(), left_on = offers.index, 
                               right_on =reactions.groupby ('person_x')[auswahl].sum().index).sort_values ('visits_count', ascending = False)
    
    return offers

In [11]:
def get_offer_influence_score (i):
    
    '''
        The influence score of each offer should tell me,
        whether one is likely influenced or is not.
        
        5 terms where calculated
        
        This one is not in the ETL Part, because it can be adjusted and has to be discussed!
    
    '''
    xx = []
    try: 
        person = received.loc [i] ['person']
        index_received = i
        offer_id = received.loc [i] ['offer_id']
        m = received_reaction_matrix.set_index ('index_received').loc [index_received] ['completed_yn']
        index_completed_return = received_reaction_matrix.set_index ('index_received').loc [index_received] ['index_completed'] -1
        '''
            If an offer is not completed, there is no need to walk through those steps
        '''        
       
    except: 
        m = 0
        index_completed_return = 0
        
    
    try: 
        '''
            This term is for those customers, that are not part of the interquartile range,
            or for another reason are dropped.
            
        '''
        profile.set_index ('id').loc [person]['count']
        m = received_reaction_matrix.set_index ('index_received').loc [index_received] ['completed_yn']
        index_completed_return = received_reaction_matrix.set_index ('index_received').loc [index_received] ['index_completed'] -1
    except:
        m = 0
        index_completed_return = 0

    
    if m == 1:
        
        '''
            Offer is seen and completed - calculating all the terms
        '''
            
        try: offers_pPerson_received = received [received ['person'] == person].count ()['person']
        except: offers_pPerson_received = 1
    
        try: offers_pPerson_viewed = viewed [viewed ['person'] == person].count ()['person']
        except: offers_pPerson_viewed = 0
            
        count_visits_each_customer = profile.set_index ('id').loc [person]['count']
    
        offer_amount =  portfolio.set_index ('id').loc [offer_id]['difficulty']
        offer_expiration = portfolio.set_index ('id').loc [offer_id]['duration']*24
        median_amount = profiles_b.set_index ('id').loc [person]['50%']
    
        off_react_time = received_reaction_matrix.set_index ('index_received').loc [index_received] ['compacttime']
        view_to_expire = received_reaction_matrix.set_index ('index_received').loc [index_received] ['viewtoexpire']
    
        visits_count = received_reaction_matrix.set_index ('index_received').loc [index_received] ['visits_count']
        
        
        '''
            Term 1: intress in offers
            Offers viewed / num visits
            
            setting a max for this value at 3
        '''
        try: 
            interest = offers_pPerson_viewed / (offers_pPerson_received * 0.75)
            interest = min (4, interest)
            interest = max (0.5, interest)
        except: freq_visits = 0
        
        
        '''
            Term 2: Percentage money more spent on the offer than median
            
            
        '''
        try: 
            overpay = (offer_amount / (median_amount * 0.7)) ** 2
            overpay = min (4, overpay)
        except: overpay = 0
        
        
        
        '''
            Term 3: consideration
            (expiration - 48) equals some buffer time
        '''
        try: 
            consideration = ((offer_expiration-48 ) / view_to_expire) ** 0.5
            consideration = min (4, consideration)
        except: consideration = 0
        
        
        
        '''
            Term 4: attractiveness
            
        '''
        try: 
            attractiveness = off_react_time / (view_to_expire * 0.3)
            attractiveness = min (4, attractiveness)
            attractiveness = max (0, attractiveness)
        except: attractiveness = 0
       
    
    
        '''
            Term 5: fulfill_by_incident
            the more visits between recognizing the offer and completing it,
            the more unattracted those visits are by the offer itself.
            
        '''
        
        try: 
            fulfill_by_incident = 1 / (1 * visits_count)
        except: fulfill_by_incident = 1
    
    
    
        try: influence_score = interest * overpay * consideration * attractiveness * fulfill_by_incident
        except: influence_score = 0
    
        influence_score_vect = [interest, overpay, consideration, attractiveness, fulfill_by_incident]

    else: 
        influence_score = 0
        influence_score_vect = [0, 0, 0, 0, 0]
    
        
    return index_completed_return, influence_score, influence_score_vect

In [12]:
received, viewed, completed, transactions = split_on_event (transcript)

NameError: name 'transcript' is not defined

In [ ]:
i = 0
stop =  len (offer_index)

influence_vect = []
influence_scal = []
influence_ind = []


offer_index = received_reaction_matrix.set_index ('index_received') [received_reaction_matrix.set_index ('index_received') ['completed_yn'] == 1].index


while i < stop:
    
    n = offer_index [i]
    
    index_completed, influence_score, influence_score_vect = get_offer_influence_score (n)
    influence_vect.append (influence_score_vect)
    influence_scal.append (influence_score)
    influence_ind.append (index_completed)
    
    
    print ('%.2f' % (i*100 / stop ), end = '\r')
    
    i += 1

In [ ]:
'''
    Clustering

'''

In [ ]:
from sklearn import preprocessing

profiles_normalized = profile
Num_features= ['age', 'income', 'member_x_weeks']
profiles_normalized[Num_features]=preprocessing.MinMaxScaler().fit_transform(profiles_normalized[Num_features])

# set the data-types:

profiles_normalized ['gender'] = profiles_normalized.gender.astype (object)
profiles_normalized ['age'] = profiles_normalized.age.astype (float)
profiles_normalized ['id'] = profiles_normalized.id.astype (object)

profiles_normalized ['F'] = profiles_normalized.F.astype (object)
profiles_normalized ['M'] = profiles_normalized.M.astype (object)
profiles_normalized ['O'] = profiles_normalized.O.astype (object)
profiles_normalized ['member_x_weeks'] = profiles_normalized.member_x_weeks.astype (float)

profiles_normalized ['income'] = profiles_normalized.income.astype (float)

In [ ]:
from dython.model_utils import roc_graph
from dython.nominal import associations

def associations_example():
    associations(Data_Correlation, nominal_columns=['age','member_x_weeks', 'income', '50%', 'count', 'std'])
    
    
# data correlation analyzis:


Data_Correlation=profiles_normalized.loc[:, ['age', 'income', 'F', 'M', 'O', 'member_x_weeks', '50%', 'count', 'std']]

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=10, 10
associations_example()

In [ ]:
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
import numpy as np
from math import isnan
 
def hopkins(X):
    d = X.shape[1]
    #d = len(vars) # columns
    n = len(X) # rows
    m = int(0.1 * n) # heuristic from article [1]
    nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
 
    rand_X = sample(range(0, n, 1), m)
 
    ujd = []
    wjd = []
    for j in range(0, m):
        u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
        ujd.append(u_dist[0][1])
        w_dist, _ = nbrs.kneighbors(X.iloc[rand_X[j]].values.reshape(1, -1), 2, return_distance=True)
        wjd.append(w_dist[0][1])
 
    H = sum(ujd) / (sum(ujd) + sum(wjd))
    if isnan(H):
        print(ujd, wjd)
        H = 0
 
    return H

In [ ]:
Data = profiles_normalized [['member_x_weeks', 'income', 'age', '50%', 'count', 'std']]

#Use a random sample of Data for faster computation
#Resetting the indexs
Data=Data.reset_index(drop=True)
#Rename the levels within in the CancellationCode column

Data.columns

#Checking whether data can be clustered
Num_features =Data.select_dtypes(include=[np.number]).columns
print ('hopkins Criteria:' hopkins(Data[Num_features]))

In [ ]:
#Principal Components converted to a Data frame
principalDf  = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'principal component 5'])
principalDf.shape


profiles_pca = pd.concat ([principalDf, profiles_normalized.reset_index()[['F', 'M', 'O', 'index']]], axis = 1)

profiles_pca['F'] = profiles_pca['F'].astype (int)
profiles_pca['M'] = profiles_pca['M'].astype (int)
profiles_pca['O'] = profiles_pca['O'].astype (int)

In [ ]:
# Attention -- time consuming calculation

from kmodes.kprototypes import KPrototypes

#Choosing optimal K value
cost = []
X = profiles_pca[['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'F', 'M', 'O']]
for num_clusters in list(range(2,10)):
    kproto = KPrototypes(n_clusters=num_clusters, init='Huang', random_state=42,n_jobs=-2,max_iter=15,n_init=50) 
    kproto.fit_predict(X, categorical=[4, 5, 6])
    cost.append(kproto.cost_)

plt.plot(cost)
plt.xlabel('K')
plt.ylabel('cost')
plt.show

In [ ]:
from kmodes.kprototypes import KPrototypes

# number of clusters
k = 10

kproto = KPrototypes(n_clusters=k, init='Huang', verbose=0, random_state=42,max_iter=20, n_init=50,n_jobs=-2,gamma=.25) 
clusters = kproto.fit_predict(profiles_pca[['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 
                                 'F', 'M', 'O']], 
                              categorical =[4, 5, 6])

df=pd.DataFrame(profiles_pca)
df['Cluster_id']=clusters
b = df [['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'Cluster_id']]
print(df['Cluster_id'].value_counts())
sns.pairplot(b,hue='Cluster_id',palette='Dark2',diag_kind='kde')

In [ ]:
def influence_scalar_to_binary (swell_value):
    
    '''
        Decide on what swell the transaction is called influenced
    '''

    i = 0

    influence_yn = []

    while i < len (influence_DF):
        if influence_DF.loc [i][0] >= swell_value:
            influence_yn.append (1)
        else:
            influence_yn.append (0)
        i += 1

    
    influence_bin = influence_DF.merge (pd.DataFrame(influence_yn), left_index = True, right_index = True)
    return influence_bin

In [ ]:
def merge_transactions_analyzes ():
    
    '''
        get all data in one DF
    
    '''
    transactions_analyzes_Df = transactions.merge (influence_bin.drop (['0_x'], axis = 1), 
                    how = 'left', left_on = transactions.index, right_on = influence_bin['index'])
    
    transactions_analyzes_Df.rename (columns = {'key_0': 'id_completed'}, inplace = True)
    DF = transactions_analyzes_Df.merge (received_reaction_matrix.set_index (['index_completed']).drop ([-1]), 
                                left_on = 'id_completed',
                                right_on = received_reaction_matrix.set_index (['index_completed']).drop ([-1]).index,
                               how = 'left')
    

    cols = ['event_x', 'value_x', 'time in hours_x', 'count_x', 'offer_id', 'DayOfWeek_x', 'WeekOfMonth_x',
           'DayOfMonth_x', 'offer_type_x', 'index', 'person_x', 'event_y', 'value_y', 'time in hours_y', 'amount_y',
           'count_y', '0b1e1539f2cc45b7b9fa7c272da2e1d7_x', '2298d6c36e964ae4a3e7e9706d1fb8c2_x',
           '2906b810c7d4411798c6938adc9daaa5_x', '3f207df678b143eea3cee63160fa8bed_x', '4d5c57ea9a6940dd891ad53e9dbe8da0_x',
            '5a8bc65990b245e5a138643cd4eb9837_x', '9b98b8c7a33c4b65b9aebfe6a799e6d9_x', 'ae264e3637204a6fb9bb56bc8210ddfd_x',
            'f19421c1d4aa40978ebb69ca19b0e20d_x', 'fafdcd668e3743c1bb461111dcafc2a4_x', 'DayOfWeek_y',
            'WeekOfMonth_y', 'DayOfMonth_y', 'expiration_y', 'offer_id_y', 'person_y']
    
    
    DF = DF.drop (cols, axis = 1)
    
    return DF

In [ ]:
def add_clusters ():
    
    '''
        Add the found clusters to the transcript
        
        99 is the cluster for not indentified persons.
    '''
    
    i = 0
    cluster = []
    stop = len (a)
    
    while i < stop:
        
        try: cluster.append (profile.set_index (['id']).loc [a.iloc [i]['person']] ['Cluster_id'])
        except: cluster.append (99)
        
        i += 1
        print ('%.2f' % (i*100 / stop ), end = '\r')
    
    return cluster





In [ ]:
def get_offer_value (cluster):
    
    '''
        This is the evaluation:
        
        For each cluster, I will look at the costs (unknown completion) 
        and the value (influenced completion = extra visit). 
    
    '''
    
    cluster_DF = analyzes_DF [analyzes_DF ['cluster'] == cluster]
    
    
    total_sales_count = cluster_DF['amount_x'].count ()
    total_sales_sum = cluster_DF['amount_x'].sum ()
    
    # completed unknown (bogo and discount)
    
    unsucc_offer_bogo_count = cluster_DF [cluster_DF ['completed_yn_unknown'] == 1][cluster_DF [cluster_DF ['completed_yn_unknown'] == 1] ['offer_type_y'] == 'bogo']['amount_x'].count ()
    unsucc_offer_bogo_sum   = cluster_DF [cluster_DF ['completed_yn_unknown'] == 1][cluster_DF [cluster_DF ['completed_yn_unknown'] == 1] ['offer_type_y'] == 'bogo']['amount_x'].sum ()
    unsucc_offer_discount_count = cluster_DF [cluster_DF ['completed_yn_unknown'] == 1][cluster_DF [cluster_DF ['completed_yn_unknown'] == 1] ['offer_type_y'] == 'discount']['amount_x'].count ()
    unsucc_offer_discount_sum   = cluster_DF [cluster_DF ['completed_yn_unknown'] == 1][cluster_DF [cluster_DF ['completed_yn_unknown'] == 1] ['offer_type_y'] == 'discount']['amount_x'].sum ()
    
    # completed (bogo and discount)
    succ_offer_bogo = cluster_DF [cluster_DF ['completed_yn'] == 1][cluster_DF [cluster_DF ['completed_yn'] == 1] ['offer_type_y'] == 'bogo']
    succ_offer_discount = cluster_DF [cluster_DF ['completed_yn'] == 1][cluster_DF [cluster_DF ['completed_yn'] == 1] ['offer_type_y'] == 'discount']
   

    # completed influenced
    influenced_bogo_count = succ_offer_bogo [succ_offer_bogo ['0_y'] == 1]['amount_x'].count ()
    influenced_bogo_sum   = succ_offer_bogo [succ_offer_bogo ['0_y'] == 1]['amount_x'].sum ()
    influenced_discount_count = succ_offer_discount [succ_offer_discount ['0_y'] == 1]['amount_x'].count ()
    influenced_discount_sum  = succ_offer_discount [succ_offer_discount ['0_y'] == 1]['amount_x'].sum ()
    
    # completed no-influence
    non_influenced_bogo_count = succ_offer_bogo [succ_offer_bogo ['0_y'] == 0]['amount_x'].count ()
    non_influenced_bogo_sum   = succ_offer_bogo [succ_offer_bogo ['0_y'] == 0]['amount_x'].sum ()
    non_influenced_discount_count = succ_offer_discount [succ_offer_discount ['0_y'] == 0]['amount_x'].count ()
    non_influenced_discount_sum = succ_offer_discount [succ_offer_discount ['0_y'] == 0]['amount_x'].sum ()
    
    
    DF_count = [total_sales_count, unsucc_offer_bogo_count, 
          unsucc_offer_discount_count, influenced_bogo_count, 
         influenced_discount_count, non_influenced_bogo_count,
         non_influenced_discount_count]
    DF_count_name = ['total_sales_count', 'unsucc_offer_bogo_count', 
          'unsucc_offer_discount_count', 'influenced_bogo_count', 
         'influenced_discount_count', 'non_influenced_bogo_count', 
         'non_influenced_discount_count']
    
    DF_sum = [total_sales_sum, unsucc_offer_bogo_sum,
          unsucc_offer_discount_sum, influenced_bogo_sum, 
         influenced_discount_sum,  non_influenced_bogo_sum ,
         non_influenced_discount_sum]
    DF_sum_name = ['total_sales_sum', 'unsucc_offer_bogo_sum',
          'unsucc_offer_discount_sum', 'influenced_bogo_sum', 
         'influenced_discount_sum', 'non_influenced_bogo_sum' ,
         'non_influenced_discount_sum']
    
    DF_count = pd.DataFrame (DF_count, index = DF_count_name)
    
    DF_sum = pd.DataFrame (DF_sum, index = DF_sum_name)
    return DF_count, DF_sum





In [ ]:
columns = ['index', 'gender', 'age', 'id', 'became_member_on',
           'income', 'count', 'mean', 'min', '50%', 'max', 'member_x_weeks',
           'F', 'M', 'O']

profile = profile[columns].merge (profiles_pca ['Cluster_id'], left_index = True, right_index = True)

profile

In [ ]:
influence_bin = influence_scalar_to_binary (0.2)



In [ ]:
transcript_analyzes = merge_transactions_analyzes ()
transcript_analyzes

In [ ]:
clusters = add_clusters ()

In [ ]:
analyzes_DF = analyzes_DF.drop ('cluster', axis = 1).merge (pd.DataFrame(clusters), left_index = True, right_index = True)

In [ ]:
analyzes_DF.rename (columns = {0: 'cluster'}, inplace = True)

In [ ]:
i = 1

while i < k:
    DF_count[i], DF_sum[i] = get_offer_value (i)
    i += 1
    
    
DF_sum.plot (kind = 'bar')

In [ ]:
DF_sum.loc ['total_sales_sum']

In [ ]:
(DF_sum/ DF_sum.loc ['total_sales_sum']).drop (['total_sales_sum'])

In [ ]:
(DF_sum/ DF_count.loc ['total_sales_count']).drop (['total_sales_sum']).plot (kind = 'bar')

In [ ]:
avg_amount_process = (DF_sum/ DF_count.loc ['total_sales_count']).drop (['total_sales_sum'])
avg_amount_process

In [ ]:
i = 0

relation_avg_amount = []
while i < avg_amount_process.shape [1]:
    relation_avg_amount.append ((avg_amount_process [i]/avg_amount_process.mean (axis = 1))-1)
    i += 1

In [ ]:
pd.DataFrame(relation_avg_amount).T.plot (kind = 'bar')

Performance against average... for total performance, we need to get rid of the bias.

# that is why it is called a quick and dirty way!

It is shown as relation + = better than average and - = less than avg. 


